<a href="https://colab.research.google.com/github/hrishipoola/Product_Pareto_Apriori_Association/blob/main/Processing_Online_Retail.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Process Online Retail Data Set

## Table of Contents
1. Introduction
2. Install & Import Packages
3. Process Data
<br> CustomerID
<br> StockCode
<br> Quantity
<br> Price
<br> Description
<br> Country
4. Add Features
5. Save
6. References

## 1. Introduction

Let's clean and process the [UCI Machine Learning Online Retail data set](https://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx) so it's ready to use later for analysis and modeling (e.g., CLTV, market basket). I've mixed, matched, and amended the cleaning logic from [here](https://github.com/koshika15/Predict-sales-of-an-online-retail-store/blob/master/A.%20Data%20Aquisition%20%26%20Wrangling.ipynb) and [here](https://github.com/amir-hojjati/Data-Analysis-Online-Retail-Transactions/blob/master/Data-Preprocessing/Preprocessing-and-Cleaning.ipynb). 

The data set includes >500,000 transactions for a UK-based online retailer that specializes in gift items with a strong wholesaler customer base. Let's process it so it's clean and  Variable definitions:  

**InvoiceNo**: 6-digit number unique to each transaction. Starting with C indicates cancellation
<br>**StockCode**: 5-digit number unique to each product
<br>**Description**: product name
<br>**Quantity**: number of each item per transaction
<br>**InvoiceDate**: date and time of transaction
<br>**UnitPrice**: price per unit in British pounds
<br>**CustomerID**: 5-digit number unique to each customer
<br>**Country**: where customer lives 

We'll walk through the logic of addressing inconsistencies in each of the variables. We'll also create additional useful features. At the end we'll have a clean, workable transactions data set to use. 

## 2. Install & Import Packages

In [131]:
import pandas as pd
import numpy as np
import datetime as dt
from datetime import date, timedelta
from datetime import date
!pip install workalendar
from workalendar.europe import UnitedKingdom
cal = UnitedKingdom()

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import style
sns.set_style("darkgrid")
style.use('fivethirtyeight')

import io
from io import BytesIO
from zipfile import ZipFile

## 3. Process Data

In [132]:
url='https://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx'
df = pd.read_excel(url,dtype= {'InvoiceNo': 'str','StockCode': 'str','CustomerID': 'str'}) # Treat identifiers as strings
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom


In [133]:
# Format dates and strings
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], format = '%m/%d/%Y %H:%M')
df['Description'] = df['Description'].str.replace('.','').str.upper().str.strip() # periods, extra white spaces
df['InvoiceNo'] = df['InvoiceNo'].str.upper()
df['StockCode'] = df['StockCode'].str.upper()
df['Country'] = df['Country'].str.upper()
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,UNITED KINGDOM
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,UNITED KINGDOM
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,UNITED KINGDOM
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,UNITED KINGDOM
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART,6,2010-12-01 08:26:00,3.39,17850,UNITED KINGDOM


In [134]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540454 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  object        
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 33.1+ MB


## CustomerID

CustomerID and Description have null values. We could derive missing CustomerIDs from InvoiceNo if CustomerIDs have multiple InvoiceNo values. Let's check the one-to-many mapping. 

In [135]:
# Check one-to-many mapping of columns to check for data correctness
def one_to_many(df, column_1, column_2):
    # Returns True if each item in column_1 is unique to each item in column_2 (first=1), but each item in column_2 may have >1 item in column_1 
    # Returns False if each item in column_1 isn't unique to each item in column_2 (first>1) and each item in column_2 has exactly 1 item in column_1 
  
    first = df.drop_duplicates([column_1, column_2]).groupby(column_1)[column_2].count().max()
    print(first == 1)

In [136]:
# Check if each InvoiceNo is unique to a CustomerID
one_to_many(df, 'InvoiceNo', 'CustomerID')

True


In [137]:
# Check if each CustomerID is unique to an InvoiceNo
one_to_many(df, 'CustomerID', 'InvoiceNo')

False


Each InvoiceNo has a unique CustomerID, but each CustomerID can have many InvoiceNo values so we can't generate missing CustomerIDs. Let's drop transactions with missing CustomerID. 


In [138]:
df = df[df.CustomerID.notnull()] # drop null Customer ID transactions 

## Stock Code


Each StockCode should have only 1 unique description,  but each description may have multiple StockCode values. But, the one_to_many function returns false, meaning some StockCode values have multiple Descriptions.

In [139]:
one_to_many(df,'StockCode','Description')

False


In [140]:
products = df.groupby(['StockCode','Description'],as_index=False).count()
products = products.loc[:,['StockCode','Description']]
stock_count = products['StockCode'].value_counts().to_frame()
stock_list = list(stock_count[stock_count.StockCode > 1].index) # List indexes that has stockCode with >1 description
products[products['StockCode'].isin(stock_list)].head(10) # filter products that have stockCode with >1 description

,StockCode,Description
42,16156L,WRAP CAROUSEL
43,16156L,"WRAP, CAROUSEL"
101,17107D,FLOWER FAIRY 5 DRAWER LINERS
102,17107D,FLOWER FAIRY 5 SUMMER DRAW LINERS
103,17107D,"FLOWER FAIRY,5 SUMMER B'DRAW LINERS"
121,20622,VIP PASSPORT COVER
122,20622,VIPPASSPORT COVER
174,20725,LUNCH BAG RED RETROSPOT
175,20725,LUNCH BAG RED SPOTTY
267,20914,SET/5 RED RETROSPOT LID GLASS BOWLS


Taking a closer look at the descriptions, it actually does look like each stock code is uniquely tied to an item, but the descriptions can vary slightly due to data entry inconsistency. Let's drop duplicates. 

In [141]:
# Remove duplicate entries
products.drop_duplicates('StockCode', keep='first', inplace=True)

In [142]:
# Correct the mapping of StockCode and Description
df = pd.merge(df, products, on='StockCode', how='left', validate='many_to_one')
df.drop('Description_x',axis=1,inplace=True)
df.rename(columns={'Description_y':'Description'},inplace=True)
df.head()

,InvoiceNo,StockCode,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Description
0,536365,85123A,6,2010-12-01 08:26:00,2.55,17850,UNITED KINGDOM,CREAM HANGING HEART T-LIGHT HOLDER
1,536365,71053,6,2010-12-01 08:26:00,3.39,17850,UNITED KINGDOM,WHITE METAL LANTERN
2,536365,84406B,8,2010-12-01 08:26:00,2.75,17850,UNITED KINGDOM,CREAM CUPID HEARTS COAT HANGER
3,536365,84029G,6,2010-12-01 08:26:00,3.39,17850,UNITED KINGDOM,KNITTED UNION FLAG HOT WATER BOTTLE
4,536365,84029E,6,2010-12-01 08:26:00,3.39,17850,UNITED KINGDOM,RED WOOLLY HOTTIE WHITE HEART


In [143]:
# Confirm that now each StockCode is unique to each description
one_to_many(df,'StockCode','Description')

True


Some StockCodes are below 5 digits. Looking at them below, they seem to be incidental charges like  postage, debt adjustments, samples, etc. and not retail sales, so let's remove them. 

In [144]:
df[(df.StockCode.str.len())<5].head(10)

,InvoiceNo,StockCode,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Description
45,536370,POST,3,2010-12-01 08:45:00,18.00,12583,FRANCE,POSTAGE
141,C536379,D,-1,2010-12-01 09:41:00,27.50,14527,UNITED KINGDOM,DISCOUNT
386,536403,POST,1,2010-12-01 11:27:00,15.00,12791,NETHERLANDS,POSTAGE
1122,536527,POST,1,2010-12-01 13:04:00,18.00,12662,GERMANY,POSTAGE
1422,536540,C2,1,2010-12-01 14:05:00,50.00,14911,EIRE,CARRIAGE
1698,536569,M,1,2010-12-01 15:35:00,1.25,16274,UNITED KINGDOM,MANUAL
1709,536569,M,1,2010-12-01 15:35:00,18.95,16274,UNITED KINGDOM,MANUAL
3868,536840,POST,1,2010-12-02 18:27:00,18.00,12738,GERMANY,POSTAGE
4053,536852,POST,1,2010-12-03 09:51:00,18.00,12686,FRANCE,POSTAGE
4120,536858,POST,2,2010-12-03 10:36:00,40.00,13520,SWITZERLAND,POSTAGE


In [145]:
df = df[(df.StockCode.str.len())>=5].reset_index(drop=True) # Drop stock codes less than 5 digits 

Some stock codes are more than 5 digits that seem to be valid, for example, an extra letter distinguishes product variations. Let's keep them.  

In [146]:
df[(df.StockCode.str.len())>5].head(10)

,InvoiceNo,StockCode,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Description
0,536365,85123A,6,2010-12-01 08:26:00,2.55,17850,UNITED KINGDOM,CREAM HANGING HEART T-LIGHT HOLDER
2,536365,84406B,8,2010-12-01 08:26:00,2.75,17850,UNITED KINGDOM,CREAM CUPID HEARTS COAT HANGER
3,536365,84029G,6,2010-12-01 08:26:00,3.39,17850,UNITED KINGDOM,KNITTED UNION FLAG HOT WATER BOTTLE
4,536365,84029E,6,2010-12-01 08:26:00,3.39,17850,UNITED KINGDOM,RED WOOLLY HOTTIE WHITE HEART
48,536373,85123A,6,2010-12-01 09:02:00,2.55,17850,UNITED KINGDOM,CREAM HANGING HEART T-LIGHT HOLDER
50,536373,84406B,8,2010-12-01 09:02:00,2.75,17850,UNITED KINGDOM,CREAM CUPID HEARTS COAT HANGER
59,536373,82494L,6,2010-12-01 09:02:00,2.55,17850,UNITED KINGDOM,WOODEN FRAME ANTIQUE WHITE
60,536373,84029G,6,2010-12-01 09:02:00,3.39,17850,UNITED KINGDOM,KNITTED UNION FLAG HOT WATER BOTTLE
61,536373,84029E,6,2010-12-01 09:02:00,3.39,17850,UNITED KINGDOM,RED WOOLLY HOTTIE WHITE HEART
65,536375,85123A,6,2010-12-01 09:32:00,2.55,17850,UNITED KINGDOM,CREAM HANGING HEART T-LIGHT HOLDER


Additionally, there are 12 transactions with StockCode 'BANK CHARGES' that all have a UnitPrice of 15. These look like charges for select customers not directly tied to sales so let's remove them. 

In [147]:
df[(df.StockCode.str.len())>7]

,InvoiceNo,StockCode,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Description
3212,536779,BANK CHARGES,1,2010-12-02 15:08:00,15.000,15823,UNITED KINGDOM,BANK CHARGES
38326,541505,BANK CHARGES,1,2011-01-18 15:58:00,15.000,15939,UNITED KINGDOM,BANK CHARGES
105494,549717,BANK CHARGES,1,2011-04-11 14:56:00,15.000,14606,UNITED KINGDOM,BANK CHARGES
122388,551945,BANK CHARGES,1,2011-05-05 11:09:00,15.000,16714,UNITED KINGDOM,BANK CHARGES
236644,565735,BANK CHARGES,1,2011-09-06 12:25:00,15.000,16904,UNITED KINGDOM,BANK CHARGES
264205,568375,BANK CHARGES,1,2011-09-26 17:01:00,15.000,13405,UNITED KINGDOM,BANK CHARGES
264206,568375,BANK CHARGES,1,2011-09-26 17:01:00,0.001,13405,UNITED KINGDOM,BANK CHARGES
302004,571900,BANK CHARGES,1,2011-10-19 14:26:00,15.000,13263,UNITED KINGDOM,BANK CHARGES
321864,573586,BANK CHARGES,1,2011-10-31 14:48:00,15.000,14704,UNITED KINGDOM,BANK CHARGES
329689,574546,BANK CHARGES,1,2011-11-04 14:59:00,15.000,13651,UNITED KINGDOM,BANK CHARGES


In [148]:
df = df[df.StockCode != 'BANK CHARGES'].reset_index(drop=True) # Drop stock codes that are 'BANK CHARGES' 

In [149]:
df.describe(include='all')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  """Entry point for launching an IPython kernel.


,InvoiceNo,StockCode,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Description
count,404909,404909,404909.000000,404909,404909.000000,404909,404909,404909
unique,21788,3676,NaN,20135,NaN,4363,37,3651
top,576339,85123A,NaN,2011-11-14 15:27:00,NaN,17841,UNITED KINGDOM,CREAM HANGING HEART T-LIGHT HOLDER
freq,541,2077,NaN,541,NaN,7971,361255,2077
first,NaN,NaN,NaN,2010-12-01 08:26:00,NaN,NaN,NaN,NaN
last,NaN,NaN,NaN,2011-12-09 12:50:00,NaN,NaN,NaN,NaN
mean,NaN,NaN,12.105799,NaN,2.901129,NaN,NaN,NaN
std,NaN,NaN,249.237543,NaN,4.430846,NaN,NaN,NaN
min,NaN,NaN,-80995.000000,NaN,0.000000,NaN,NaN,NaN
25%,NaN,NaN,2.000000,NaN,1.250000,NaN,NaN,NaN


As expected, StockCode has more unique values than description 

Needs further investigation: 
- Negative quantity
- Minimum price is 0

## Quantity

Canceled orders have InvoiceNo that starts with C. Check if negative quantities are due to canceled orders.

In [150]:
print('Number of negative quantities that are not canceled items:', len(df[(df.Quantity < 0) & (df.InvoiceNo.str[0]!='C')])) 

Number of negative quantities that are not canceled items: 0


All negative quantities are canceled items. 



In [151]:
# Check quantities over 2000
df[(abs(df.Quantity)>2000)]

,InvoiceNo,StockCode,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Description
3095,C536757,84347,-9360,2010-12-02 14:23:00,0.03,15838,UNITED KINGDOM,ROTATING SILVER ANGELS T-LIGHT HLDR
3732,536830,84077,2880,2010-12-02 17:38:00,0.18,16754,UNITED KINGDOM,WORLD WAR 2 GLIDERS ASSTD DESIGNS
22146,539101,22693,2400,2010-12-16 10:35:00,0.94,16029,UNITED KINGDOM,GROW A FLYTRAP OR SUNFLOWER IN TIN
33497,540815,21108,3114,2011-01-11 12:55:00,2.10,15749,UNITED KINGDOM,FAIRY CAKE FLANNEL ASSORTED COLOUR
37949,541431,23166,74215,2011-01-18 10:01:00,1.04,12346,UNITED KINGDOM,MEDIUM CERAMIC TOP STORAGE JAR
37954,C541433,23166,-74215,2011-01-18 10:17:00,1.04,12346,UNITED KINGDOM,MEDIUM CERAMIC TOP STORAGE JAR
50920,543057,84077,2592,2011-02-03 10:50:00,0.21,16333,UNITED KINGDOM,WORLD WAR 2 GLIDERS ASSTD DESIGNS
55636,543669,22693,2400,2011-02-11 11:22:00,0.94,16029,UNITED KINGDOM,GROW A FLYTRAP OR SUNFLOWER IN TIN
58917,544152,18007,2400,2011-02-16 12:10:00,0.06,14609,UNITED KINGDOM,ESSENTIAL BALM 35G TIN IN ENVELOPE
63033,544612,22053,3906,2011-02-22 10:43:00,0.82,18087,UNITED KINGDOM,EMPIRE DESIGN ROSETTE


- Quantities 3114, 74215, and 80995 are tied to matching canceled orders

In [152]:
print('{:.2f}% of orders were canceled'.format(len(df[df.InvoiceNo.str[0]=='C']) / len(df) * 100))

2.11% of orders were canceled


We'll create a separate dataframe for canceled orders in case we want to explore them later. 

Ideally, we should also remove the original purchases that were subsequently canceled, but I wasn't able to find a way to any pattern or matching logic with StockCode (which are many to one with each item) or InvoiceNo. 

## Price

There are also prices equal to 0, let's drop these. 

In [ ]:
df[df.UnitPrice <= 0]

,InvoiceNo,StockCode,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Description
6986,537197,22841,1,2010-12-05 14:02:00,0.0,12647,GERMANY,ROUND CAKE TIN VINTAGE GREEN
22988,539263,22580,4,2010-12-16 14:36:00,0.0,16560,UNITED KINGDOM,ADVENT CALENDAR GINGHAM SACK
25937,539722,22423,10,2010-12-21 13:45:00,0.0,14911,EIRE,REGENCY CAKESTAND 3 TIER
29750,540372,22090,24,2011-01-06 16:41:00,0.0,13081,UNITED KINGDOM,PAPER BUNTING RETROSPOT
29752,540372,22553,24,2011-01-06 16:41:00,0.0,13081,UNITED KINGDOM,PLASTERS IN TIN SKULLS
35300,541109,22168,1,2011-01-13 15:10:00,0.0,15107,UNITED KINGDOM,ORGANISER WOOD ANTIQUE WHITE
55025,543599,84535B,16,2011-02-10 13:08:00,0.0,17560,UNITED KINGDOM,FAIRY CAKES NOTEBOOK A6 SIZE
87481,547417,22062,36,2011-03-23 10:25:00,0.0,13239,UNITED KINGDOM,CERAMIC BOWL WITH LOVE HEART DESIGN
94745,548318,22055,5,2011-03-30 12:45:00,0.0,13113,UNITED KINGDOM,MINI CAKE STAND HANGING STRAWBERY
99450,548871,22162,2,2011-04-04 14:42:00,0.0,14410,UNITED KINGDOM,HEART GARLAND RUSTIC PADDED


In [ ]:
df = df[df.UnitPrice > 0].reset_index(drop=True) # Keep only positive prices 

Each StockCode should have a unique UnitPrice, but the one_to_many function shows that some StockCode values have multiple UnitPrice values. This could be due to discounts and coupons or promotions on specific days. To correct this, we'll set the unit price for each StockCode to its mode UnitPrice. 

In [ ]:
one_to_many(df,'StockCode','UnitPrice')

False


In [ ]:
prices = df.groupby(['StockCode','UnitPrice'],as_index=False).count()
prices = prices.loc[:,['StockCode','UnitPrice']]
price_count = prices['StockCode'].value_counts().to_frame()
price_list = list(price_count[price_count.StockCode > 1].index) # List indexes that has stockCode with >1 UnitPrice
prices[prices['StockCode'].isin(price_list)] # filter products that have stockCode with >1 UnitPrice

,StockCode,UnitPrice
1,10080,0.39
2,10080,0.85
7,10125,0.42
8,10125,0.85
9,10133,0.42
...,...,...
8461,90214S,1.25
8464,90214V,0.29
8465,90214V,1.25
8467,90214Y,0.29


In [ ]:
stock_list = df.StockCode.unique()
price_mode = map(lambda x: df.UnitPrice[df.StockCode == x].mode()[0],stock_list)
stock_mode = list(price_mode)
for i,v in enumerate(stock_list):
    df.loc[df['StockCode']== v, 'UnitPrice'] = stock_mode[i]

In [ ]:
one_to_many(df,'StockCode','UnitPrice')

True


## Description

In [ ]:
print('Number of empty descriptions:',len(df[df.Description.isna()==True])) 

Number of empty descriptions: 0


## Country

Each CustomerID should be tied to 1 country, but we see below that a handful of CustomerIDs are linked to more than 1 country. It's unclear whether the countries are captured by IP address at purchase or shipping or billing details. Additionally, customers may have moved, purchased while traveling, or shipped to another address from their billing. To treat this, for CustomerIDs with multiple countries, we'll replace countries with the customer's mode country.   

In [ ]:
one_to_many(df,'CustomerID','Country')

False


In [ ]:
# Customers with >1 country 
customer_country = df.groupby('CustomerID')['Country'].unique()
customer_country.loc[customer_country.apply(lambda x:len(x)>1)]

CustomerID
12370           [CYPRUS, AUSTRIA]
12394          [BELGIUM, DENMARK]
12417            [BELGIUM, SPAIN]
12422    [AUSTRALIA, SWITZERLAND]
12429          [DENMARK, AUSTRIA]
12431        [AUSTRALIA, BELGIUM]
12455             [CYPRUS, SPAIN]
12457       [SWITZERLAND, CYPRUS]
Name: Country, dtype: object

In [ ]:
# For customer's with multiple countries, replace with mode 
customers = df.groupby('CustomerID')['Country'].unique().items()
for i,v in customers:
    if len(v)>1:
        df.Country[df['CustomerID'] == i] = df.Country[df['CustomerID'] == i].mode()[0]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
one_to_many(df,'CustomerID','Country')

True


In [ ]:
df.Country.unique()

array(['UNITED KINGDOM', 'FRANCE', 'AUSTRALIA', 'NETHERLANDS', 'GERMANY',
       'NORWAY', 'EIRE', 'SWITZERLAND', 'SPAIN', 'POLAND', 'PORTUGAL',
       'ITALY', 'BELGIUM', 'LITHUANIA', 'JAPAN', 'ICELAND',
       'CHANNEL ISLANDS', 'DENMARK', 'CYPRUS', 'SWEDEN', 'AUSTRIA',
       'ISRAEL', 'FINLAND', 'GREECE', 'SINGAPORE', 'LEBANON',
       'UNITED ARAB EMIRATES', 'SAUDI ARABIA', 'CZECH REPUBLIC', 'CANADA',
       'UNSPECIFIED', 'BRAZIL', 'USA', 'EUROPEAN COMMUNITY', 'BAHRAIN',
       'MALTA', 'RSA'], dtype=object)

In [ ]:
# Filter and format countries 
df.query('Country not in ["UNSPECIFIED", "EUROPEAN COMMUNITY"]', inplace=True)
df.reset_index(inplace=True, drop=True)
df.Country.replace({'EIRE': 'IRELAND', 'RSA': 'SOUTH AFRICA'}, inplace=True)

In [ ]:
df.Country.value_counts()

UNITED KINGDOM          361235
GERMANY                   9095
FRANCE                    8168
IRELAND                   7378
SPAIN                     2486
NETHERLANDS               2326
BELGIUM                   1946
SWITZERLAND               1829
PORTUGAL                  1436
AUSTRALIA                 1323
NORWAY                    1059
ITALY                      783
CHANNEL ISLANDS            753
FINLAND                    653
CYPRUS                     582
SWEDEN                     437
DENMARK                    390
AUSTRIA                    360
JAPAN                      355
POLAND                     336
USA                        291
ISRAEL                     250
SINGAPORE                  215
ICELAND                    182
CANADA                     150
GREECE                     142
MALTA                      123
UNITED ARAB EMIRATES        67
SOUTH AFRICA                57
LEBANON                     45
LITHUANIA                   35
BRAZIL                      32
CZECH RE

## 4. Add Features

In [ ]:
# Create separate dataframe for canceled orders in case we want to investigate them later 
canceled_orders = df[df.InvoiceNo.str[0] == 'C']
canceled_orders.reset_index(inplace=True, drop=True)

df = df[df.InvoiceNo.str[0] != 'C']
df.reset_index(inplace=True, drop=True)

In [ ]:
df = df.assign(TotalPrice = df.Quantity * df.UnitPrice,
               Year = lambda x: x.InvoiceDate.dt.year,
               Month = lambda x: x.InvoiceDate.dt.month,
               Day = lambda x: x.InvoiceDate.dt.day,
               DayName = lambda x: x.InvoiceDate.dt.day_name(), 
               Hour = lambda x: x.InvoiceDate.dt.hour)

# Holiday variable
start = df.InvoiceDate.min() 
start_year = df.Year.min() 
end = df.InvoiceDate.max() 
end_year = df.Year.max() 

holidays = set(holiday[0] 
               for year in range(start_year, end_year + 1)
               for holiday in cal.holidays(year)
               if start.date() <= holiday[0] <= end.date())

df = df.assign(Holiday = df.InvoiceDate.dt.date.isin(holidays).astype(int))
df.head()

,InvoiceNo,StockCode,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Description,TotalPrice,Year,Month,Day,DayName,Hour,Holiday
0,536365,85123A,6,2010-12-01 08:26:00,2.95,17850,UNITED KINGDOM,CREAM HANGING HEART T-LIGHT HOLDER,17.7,2010,12,1,Wednesday,8,0
1,536365,71053,6,2010-12-01 08:26:00,3.75,17850,UNITED KINGDOM,WHITE METAL LANTERN,22.5,2010,12,1,Wednesday,8,0
2,536365,84406B,8,2010-12-01 08:26:00,4.15,17850,UNITED KINGDOM,CREAM CUPID HEARTS COAT HANGER,33.2,2010,12,1,Wednesday,8,0
3,536365,84029G,6,2010-12-01 08:26:00,4.25,17850,UNITED KINGDOM,KNITTED UNION FLAG HOT WATER BOTTLE,25.5,2010,12,1,Wednesday,8,0
4,536365,84029E,6,2010-12-01 08:26:00,4.25,17850,UNITED KINGDOM,RED WOOLLY HOTTIE WHITE HEART,25.5,2010,12,1,Wednesday,8,0


In [ ]:
# Check UK holidays in 2011
cal.holidays(2011)

[(datetime.date(2011, 1, 1), 'New year'),
 (datetime.date(2011, 1, 3), 'New Year shift'),
 (datetime.date(2011, 4, 22), 'Good Friday'),
 (datetime.date(2011, 4, 24), 'Easter Sunday'),
 (datetime.date(2011, 4, 25), 'Easter Monday'),
 (datetime.date(2011, 4, 29), 'Royal Wedding'),
 (datetime.date(2011, 5, 2), 'Early May Bank Holiday'),
 (datetime.date(2011, 5, 30), 'Spring Bank Holiday'),
 (datetime.date(2011, 8, 29), 'Late Summer Bank Holiday'),
 (datetime.date(2011, 12, 25), 'Christmas Day'),
 (datetime.date(2011, 12, 26), 'Boxing Day'),
 (datetime.date(2011, 12, 26), 'Christmas Shift'),
 (datetime.date(2011, 12, 27), 'Boxing Day Shift')]

## 5. Save

In [ ]:
# Mount google drive
from google.colab import drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 396036 entries, 0 to 396035
Data columns (total 15 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    396036 non-null  object        
 1   StockCode    396036 non-null  object        
 2   Quantity     396036 non-null  int64         
 3   InvoiceDate  396036 non-null  datetime64[ns]
 4   UnitPrice    396036 non-null  float64       
 5   CustomerID   396036 non-null  object        
 6   Country      396036 non-null  object        
 7   Description  396036 non-null  object        
 8   TotalPrice   396036 non-null  float64       
 9   Year         396036 non-null  int64         
 10  Month        396036 non-null  int64         
 11  Day          396036 non-null  int64         
 12  DayName      396036 non-null  object        
 13  Hour         396036 non-null  int64         
 14  Holiday      396036 non-null  int64         
dtypes: datetime64[ns](1), float64(2), 

In [94]:
# Save processed data to csv (df is too large so I zipped it)
df.to_csv('online_retail_processed.csv.gz', compression='gzip', date_format = '%Y-%m-%d %H:%M', index=False)

In [ ]:
# Save canceled orders csv
canceled_orders.to_csv('canceled_orders.csv', date_format = '%Y-%m-%d %H:%M', index=False)

## 6. References


https://stackoverflow.com/questions/57403835/pandas-using-workalendar-to-check-if-a-date-is-a-holiday-and-assigning-boolean

https://github.com/koshika15/Predict-sales-of-an-online-retail-store/blob/master/A.%20Data%20Aquisition%20%26%20Wrangling.ipynb

https://github.com/koshika15/Predict-sales-of-an-online-retail-store/blob/master/Capstone%20Project%202_%20Final%20Report.pdf

https://github.com/amir-hojjati/Data-Analysis-Online-Retail-Transactions/blob/master/Data-Preprocessing/Preprocessing-and-Cleaning.ipynb
